In [3]:
import os
import torch
import ray
from ray.util.scheduling_strategies import PlacementGroupSchedulingStrategy

# Ray 초기화
# ray.stop() 대신
if ray.is_initialized():
    ray.shutdown()
ray.init(dashboard_port=8265)

from ray.util.placement_group import placement_group
# Ray 초기화
ray.start(dashboard_port=8265)

# 환경 변수 설정
os.environ["TRANSFORMERS_NO_FLASH_ATTN"] = "1"
os.environ["RAY_MAX_RUNTIME_ENV_SIZE_BYTES"] = "500000000"

# 경로 설정
ROOT_DIR = "/path/to/your/root"  # 실제 경로로 변경하세요
MODEL_DIR = "/path/to/your/models"  # 실제 경로로 변경하세요
BACKBONE = "Qwen/Qwen3-0.6B"  # Qwen3-0.6B 모델 사용
BACKBONE_PATH = f"{MODEL_DIR}/{BACKBONE}"

# 출력 디렉토리 설정
import datetime
DATE = datetime.datetime.now().strftime("%m%d")
EXPERIMENT = "TTRL"
ADVANTAGE = "group_norm"
TASK = "open_r1"
MODEL = f"{TASK}-{BACKBONE}"
OUTPUT_DIR = f"{ROOT_DIR}/outputs/{MODEL}/{DATE}/{EXPERIMENT}-{ADVANTAGE}"

# 필요한 디렉토리 생성
os.makedirs(f"{ROOT_DIR}/logs", exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# PPO 학습 실행
print("PPO 학습 시작...")

# vLLM 설정
vllm_num_engines = 1  # 사용 가능한 GPU 수에 따라 조정
vllm_tensor_parallel_size = 1

# 학습 명령어 실행
# 실제 환경에 맞게 파라미터 조정
cmd = f"""
python -m ttrl.cli.train_ppo_naive \
  --ref_num_nodes 1 \
  --ref_num_gpus_per_node 1 \
  --critic_num_nodes 1 \
  --critic_num_gpus_per_node 1 \
  --actor_num_nodes 1 \
  --actor_num_gpus_per_node 1 \
  --vllm_num_engines {vllm_num_engines} \
  --vllm_tensor_parallel_size {vllm_tensor_parallel_size} \
  --colocate_actor_ref \
  --pretrain "{BACKBONE_PATH}" \
  --save_path "{OUTPUT_DIR}/model" \
  --batch_size 4 \
  --mini_batch_size 2 \
  --epochs 3 \
  --lr 1e-5 \
  --kl_coef 0.1 \
  --clip_range_value 0.2
"""

print(f"실행 명령어: {cmd}")

# 실행 방법 안내
print("""
참고: 이 코드는 노트북 셀에서 직접 실행되지 않을 수 있습니다.
다음 방법으로 실행하세요:
1. 터미널에서 위 명령어를 실행하거나
2. 아래 코드를 사용하여 subprocess로 실행
""")

# subprocess로 실행하는 예시
import subprocess

def run_ppo_training():
    try:
        process = subprocess.Popen(
            cmd, 
            shell=True, 
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        # 실시간으로 출력 표시
        for line in process.stdout:
            print(line, end='')
            
        process.wait()
        return process.returncode
    except Exception as e:
        print(f"오류 발생: {e}")
        return 1

# 실행하려면 아래 주석을 해제하세요
# return_code = run_ppo_training()
# print(f"실행 완료. 반환 코드: {return_code}")


2025-05-07 22:35:56,704	INFO worker.py:1694 -- Connecting to existing Ray cluster at address: 155.230.28.29:6379...


2025-05-07 22:35:56,709	INFO worker.py:1879 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


RuntimeError: Version mismatch: The cluster was started with:
    Ray: 2.42.0
    Python: 3.10.16
This process on node 155.230.28.29 was started with:
    Ray: 2.45.0
    Python: 3.10.16
